In [3]:
import torch
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import lightning.pytorch as pl
from sklearn.model_selection import train_test_split
import numpy as np
import random


class MRIDataset(Dataset):
    def __init__(self, dataframe, slice_number, transform=None):
        self.df = dataframe.set_index(['ID', 'slice_number'])
        self.slice_number = slice_number
        self.transform = transform
        self.resize = transforms.Resize((224, 224))  # Ensure images are resized to 224x224

    def __len__(self):
        return len(self.df.index.unique())

    def __getitem__(self, idx):
        id, slice_num = self.df.index.unique()[idx]
        images = []
        for offset in (-1, 0, 1):
            slice_path = self.get_random_path(id, slice_num + offset)
            image = Image.open(slice_path).convert('L')
            image = self.resize(image)  # Resize the image
            image = np.array(image)
            images.append(image)
        
        # Stack to create a 3-channel image
        image_stack = np.stack(images, axis=-1)  # Shape will be (H, W, C)
        image_stack = torch.tensor(image_stack).permute(2, 0, 1)  # Convert to (C, H, W) tensor
        
        # Normalize the image stack to [0, 1]
        image_stack = image_stack.float() / 255.0
        
        if self.transform:
            image_stack = self.transform(image_stack)
            
        return image_stack

    def get_random_path(self, id, slice_num):
        try:
            rows = self.df.loc[(id, slice_num)]
            if not rows.empty:
                # Randomly select between masked and unmasked if available
                row = rows.sample(n=1)
                return row['path'].values[0]
            else:
                # If the specific slice_num doesn't exist, default to the original slice
                return self.df.loc[(id, self.slice_number)].sample(n=1)['path'].values[0]
        except KeyError:
            # In case the slice is completely unavailable, use the fallback slice
            return self.df.loc[(id, self.slice_number)].sample(n=1)['path'].values[0]


class MRIImageDataModule(pl.LightningDataModule):
    def __init__(self, data_path, batch_size=32, slice_number=87):
        super().__init__()
        self.data_path = data_path
        self.batch_size = batch_size
        self.slice_number = slice_number
        self.transform = transforms.Compose([
            transforms.Normalize(mean=[0.5], std=[0.5])
        ])

    def setup(self, stage=None):
        data = pd.read_csv(self.data_path)
        train_ids, test_ids = train_test_split(data['ID'].unique(), test_size=0.2, random_state=42)

        train_df = data[data['ID'].isin(train_ids)]
        test_df = data[data['ID'].isin(test_ids)]

        self.train_dataset = MRIDataset(train_df, self.slice_number, transform=self.transform)
        self.test_dataset = MRIDataset(test_df, self.slice_number, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False)


# Usage
data_module = MRIImageDataModule(data_path='Data/metadata_for_preprocessed_files.csv', slice_number=63)
data_module.setup()
train_loader = data_module.train_dataloader()
for batch in train_loader:
    # Each batch should have the dimensions [batch_size, channels, height, width]
    print(batch.shape)  # Should output torch.Size([batch_size, 3, 224, 224])
    break


torch.Size([32, 3, 224, 224])


/var/folders/l5/4bf7qdjd6k9b38g7jlnrcf580000gn/T/ipykernel_30646/2326577472.py:46: PerformanceWarning: indexing past lexsort depth may impact performance.
  rows = self.df.loc[(id, slice_num)]


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = {
    'ID': ['OAS1_0001_MR1', 'OAS1_0001_MR1', 'OAS1_0001_MR1', 'OAS1_0002_MR1', 'OAS1_0002_MR1', 'OAS1_0003_MR1'],
    'slice_number': [62, 63, 64, 63, 64, 63],
    'path': [
        'Data/OASIS_Extracted/OAS1_0001/OAS1_0001_MR1_mpr_n4_anon_111_t88_gfc_slice_62.jpeg',
        'Data/OASIS_Extracted/OAS1_0001/OAS1_0001_MR1_mpr_n4_anon_111_t88_gfc_slice_63.jpeg',
        'Data/OASIS_Extracted/OAS1_0001/OAS1_0001_MR1_mpr_n4_anon_111_t88_gfc_slice_64.jpeg',
        'Data/OASIS_Extracted/OAS1_0002/OAS1_0002_MR1_mpr_n4_anon_111_t88_gfc_slice_63.jpeg',
        'Data/OASIS_Extracted/OAS1_0002/OAS1_0002_MR1_mpr_n4_anon_111_t88_gfc_slice_64.jpeg',
        'Data/OASIS_Extracted/OAS1_0003/OAS1_0003_MR1_mpr_n4_anon_111_t88_gfc_slice_63.jpeg'
    ],
    'CDR': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    'is_masked': [True, False, True, False, True, False]
}

df = pd.DataFrame(data)

train_ids, test_ids = train_test_split(df['ID'].unique(), test_size=0.2, random_state=42)
train_df = df[df['ID'].isin(train_ids)]
test_df = df[df['ID'].isin(test_ids)]

train_size = len(train_df)
test_size = len(test_df)

train_ids, test_ids, train_size, test_size


(array(['OAS1_0002_MR1', 'OAS1_0003_MR1'], dtype=object),
 array(['OAS1_0001_MR1'], dtype=object),
 3,
 3)

In [2]:
train_df

,ID,slice_number,path,CDR,is_masked
3,OAS1_0002_MR1,63,Data/OASIS_Extracted/OAS1_0002/OAS1_0002_MR1_m...,0.0,False
4,OAS1_0002_MR1,64,Data/OASIS_Extracted/OAS1_0002/OAS1_0002_MR1_m...,0.0,True
5,OAS1_0003_MR1,63,Data/OASIS_Extracted/OAS1_0003/OAS1_0003_MR1_m...,0.0,False


In [3]:
import torch
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import lightning.pytorch as pl
from sklearn.model_selection import train_test_split
import numpy as np
import random


# class MRIDataset(Dataset):
#     def __init__(self, dataframe, slice_number, transform=None):
#         self.df = dataframe.set_index(['ID', 'slice_number'])
#         self.slice_number = slice_number
#         self.transform = transform
#         self.resize = transforms.Resize((224, 224))  # Ensure images are resized to 224x224 # TODO: better to have an assert to save time, since data should be this size already?

#     def __len__(self):
#         return len(self.df.index.unique())

#     def __getitem__(self, idx):
#         id, slice_num = self.df.index.unique()[idx]
#         images = []
#         for offset in (-1, 0, 1):
#             slice_path = self.get_random_path(id, slice_num + offset)
#             image = Image.open(slice_path).convert('L')
#             image = np.array(image)
#             images.append(image)
        
#         # Stack to create a 3-channel image
#         image_stack = np.stack(images, axis=-1)  # Shape will be (H, W, C)
#         image_stack = torch.tensor(image_stack).permute(2, 0, 1)  # Convert to (C, H, W) tensor
        
#         # Normalize the image stack to [0, 1]
#         image_stack = image_stack.float() / 255.0 # TODO: check if images were already normalized and resized to 224 by 224
        
#         if self.transform:
#             image_stack = self.transform(image_stack)
            
#         return image_stack

#     def get_random_path(self, id, slice_num):
#         try:
#             rows = self.df.loc[(id, slice_num)]
#             if not rows.empty:
#                 # Randomly select between masked and unmasked if available
#                 row = rows.sample(n=1)
#                 return row['path'].values[0]
#             else:
#                 # If the specific slice_num doesn't exist, default to the original slice
#                 return self.df.loc[(id, self.slice_number)].sample(n=1)['path'].values[0]
#         except KeyError:
#                print(f"KeyError: The slice number {slice_num} or id {id} does not exist in the Data.")
#         return None

# class MRIImageDataModule(pl.LightningDataModule):
#     def __init__(self, data_path, batch_size=32, slice_number=87):
#         super().__init__()
#         self.data_path = data_path
#         self.batch_size = batch_size
#         self.slice_number = slice_number

#     def setup(self, stage=None):
#         data = pd.read_csv(self.data_path)
#         train_ids, test_ids = train_test_split(data['ID'].unique(), test_size=0.2, random_state=42)

#         train_df = data[data['ID'].isin(train_ids)]
#         test_df = data[data['ID'].isin(test_ids)]

#         self.train_dataset = MRIDataset(train_df, self.slice_number)
#         self.test_dataset = MRIDataset(test_df, self.slice_number)

#     def train_dataloader(self):
#         return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

#     def val_dataloader(self):
#         return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False)


# Usage
# data_module = MRIImageDataModule(data_path='Data/metadata_for_preprocessed_files.csv', slice_number=63)
# data_module.setup()
# train_loader = data_module.train_dataloader()
# for batch in train_loader:
#     # Each batch should have the dimensions [batch_size, channels, height, width]
#     print(batch.shape)  # Should output torch.Size([batch_size, 3, 224, 224])
#     break


import torch
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split

class MRIDataset(Dataset):
    def __init__(self, dataframe, slice_number, transform=None):
        self.slice_number = slice_number
        self.transform = transform
        self.df = dataframe
        self.valid_ids = dataframe[dataframe['slice_number'] == slice_number]['ID'].unique()
        self.df = self.df[self.df['ID'].isin(self.valid_ids)].set_index(['ID', 'slice_number'])

    def __len__(self):
        return len(self.valid_ids)

    def __getitem__(self, idx):
        id = self.valid_ids[idx]
        images = []
        
        for offset in (-1, 0, 1):
            slice_path = self.get_random_path(id, self.slice_number + offset)
            image = Image.open(slice_path).convert('L')
            image = np.array(image)
            images.append(image)
        
        # Stack to create a 3-channel image
        image_stack = np.stack(images, axis=-1)  # Shape will be (H, W, C)
        image_stack = torch.tensor(image_stack).permute(2, 0, 1)  # Convert to (C, H, W) tensor
        
        # Normalize the image stack to [0, 1]
        image_stack = image_stack.float() / 255.0 #TODO: check if images are already normalized and sized to 224 224
        
        if self.transform:
            image_stack = self.transform(image_stack)
            
        return image_stack
    
    def get_random_path(self, id, slice_num):
        try:
            rows = self.df.loc[(id, slice_num)]
            if not rows.empty:
                # Randomly select between masked and unmasked if available
                row = rows.sample(n=1)
                return row['path'].values[0]
            else:
                # If the specific slice_num doesn't exist, default to the original slice
                return self.df.loc[(id, self.slice_number)].sample(n=1)['path'].values[0]
        except KeyError:
            # In case the slice is completely unavailable, use the fallback slice
            try: 
                return self.df.loc[(id, self.slice_number)].sample(n=1)['path'].values[0]
            except KeyError: #                
                print(f"KeyError: The slice number {self.slice_number} or id {id} does not exist in the Data.")
        return None

class MRIImageDataModule(pl.LightningDataModule):
    def __init__(self, data_path, batch_size=32, slice_number=87):
        super().__init__()
        self.data_path = data_path
        self.batch_size = batch_size
        self.slice_number = slice_number

    def setup(self, stage=None):
        data = pd.read_csv(self.data_path)
        
        # Filter to only include IDs with the specified slice_number
        data = data[data['slice_number'].isin([self.slice_number - 1, self.slice_number, self.slice_number + 1])]

        train_ids, test_ids = train_test_split(data['ID'].unique(), test_size=0.2, random_state=42)

        train_df = data[data['ID'].isin(train_ids)]
        test_df = data[data['ID'].isin(test_ids)]

        self.train_dataset = MRIDataset(train_df, self.slice_number)
        self.test_dataset = MRIDataset(test_df, self.slice_number)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False)

# Usage example
# data_module = MRIImageDataModule(data_path='Data/metadata_for_preprocessed_files.csv', slice_number=63)
# data_module.setup()
# train_loader = data_module.train_dataloader()
# for batch in train_loader:
#     print(batch.shape)  # Should output torch.Size([batch_size, 3, 224, 224])


/var/folders/l5/4bf7qdjd6k9b38g7jlnrcf580000gn/T/ipykernel_31166/2985792169.py:134: PerformanceWarning: indexing past lexsort depth may impact performance.
  rows = self.df.loc[(id, slice_num)]


                            Unnamed: 0 M/F Hand  Age  Educ  SES  MMSE  CDR  \
ID            slice_number                                                   
OAS1_0410_MR1 62                   372   F    R   23   NaN  NaN   NaN  0.0   
              62                   372   F    R   23   NaN  NaN   NaN  0.0   

                            eTIV  nWBV    ASF  Delay  \
ID            slice_number                             
OAS1_0410_MR1 62            1507  0.87  1.165    NaN   
              62            1507  0.87  1.165    NaN   

                                                                         path  \
ID            slice_number                                                      
OAS1_0410_MR1 62            Data/OASIS_Extracted/OAS1_0410/OAS1_0410_MR1_m...   
              62            Data/OASIS_Extracted/OAS1_0410/OAS1_0410_MR1_m...   

                            is_masked  
ID            slice_number             
OAS1_0410_MR1 62                False  
              62 

: 

In [1]:
from transformers import MobileViTImageProcessor
from PIL import Image
import torch

# Load the image processor
processor = MobileViTImageProcessor.from_pretrained("apple/mobilevit-xx-small")

# Load an image
image_path = "Data/Mean_Images/average_slice_0_masked_False.jpeg"
image = Image.open(image_path).convert("RGB")

# Preprocess the image
inputs = processor(images=image, return_tensors="pt")

print(inputs)


/Users/henrismidt/anaconda3/envs/alzheimer/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

{'pixel_values': tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])}


In [6]:
print(inputs["pixel_values"].size())

torch.Size([1, 3, 256, 256])


In [3]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.datasets import CIFAR10
import pytorch_lightning as pl
from transformers import MobileViTForImageClassification, MobileViTImageProcessor

# Custom dataset class to filter CIFAR-10 classes
class CIFAR10Subset(Dataset):
    def __init__(self, root, train=True, transform=None, download=False, class_subset=None):
        self.dataset = CIFAR10(root=root, train=train, transform=transform, download=download)
        self.class_subset = class_subset
        if class_subset is not None:
            self.indices = [i for i, label in enumerate(self.dataset.targets) if label in class_subset]
        else:
            self.indices = list(range(len(self.dataset)))

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        index = self.indices[idx]
        image, label = self.dataset[index]
        label = self.class_subset.index(label)  # Reindex the label to [0, 1, 2, 3]
        return image, label

# Define the LightningModule
class MobileViTLightning(pl.LightningModule):
    def __init__(self, model_ckpt, num_labels):
        super(MobileViTLightning, self).__init__()
        self.model = MobileViTForImageClassification.from_pretrained(model_ckpt, num_labels=num_labels, ignore_mismatched_sizes=True)

    def forward(self, x):
        return self.model(x).logits

    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=2e-5)

# Load the pretrained model and processor
model_ckpt = "apple/mobilevit-xx-small"
processor = MobileViTImageProcessor.from_pretrained(model_ckpt)

# Load and preprocess the CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Use only the first 4 classes for demonstration
class_subset = [0, 1, 2, 3]
train_dataset = CIFAR10Subset(root="./data", train=True, download=True, transform=transform, class_subset=class_subset)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# Initialize the model and trainer
model = MobileViTLightning(model_ckpt=model_ckpt, num_labels=4)
trainer = pl.Trainer(max_epochs=3)

# Train the model
trainer.fit(model, train_loader)


Files already downloaded and verified


Some weights of MobileViTForImageClassification were not initialized from the model checkpoint at apple/mobilevit-xx-small and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 320]) in the checkpoint and torch.Size([4, 320]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/henrismidt/anaconda3/envs/alzheimer/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML eco

Training: |          | 0/? [00:00<?, ?it/s]